In [17]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import gc
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import chi2

In [2]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # Create feature index
    cor_feature_index=np.argsort(np.abs(cor_list))[-num_feats:]
    # feature name
    cor_feature = X.iloc[:,cor_feature_index].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature, cor_feature_index, cor_list

In [3]:
# Read from data file
df_irr = pd.read_csv("gxePlantHeight.csv")
df_snp = pd.read_csv('snp_irrigation.csv')

# Preprocess
df_irr['Female'] = [x.replace('-','_').replace(' ','') for x in df_irr.Female]
df_irr['Male'] = [x.replace('-','_').replace(' ','') for x in df_irr.Male]
df_snp['Female']=[x.split(".")[0] for x in df_snp.Genotype]
df_snp['Male']=[x.split(".")[1] for x in df_snp.Genotype]

# Drop unnecessary columns
df_snp.drop('Genotype', axis=1, inplace=True)
df_irr.drop(['PID','DaysOfYear','Date','State','Country','Height-Diff','Precipitation','Humidity','SolarRadiation','Temperature','GDD','CGDD','col'], axis=1, inplace=True)

# Inner join
df = pd.merge(left=df_irr, right=df_snp, left_on=['Female','Male'], right_on=['Female','Male'])
df.head()

# Delete tables
del df_irr, df_snp
gc.collect()

56

In [4]:
df['DAP'].describe()

count    5530.000000
mean       35.500000
std        20.207024
min         1.000000
25%        18.000000
50%        35.500000
75%        53.000000
max        70.000000
Name: DAP, dtype: float64

In [5]:
days=np.arange(1,70,1)
filter_days=[[i for i in df["DAP"]==j] for j in days]

In [6]:
df1 = pd.DataFrame(df[filter_days[0]])

In [7]:
df1.head()

,DAP,Genotype,Female,Male,GrowthRate-Diff,chr1_2111,chr1_6984,chr1_7035,chr1_7080,chr1_9405,...,chr10_149808541,chr10_149811649,chr10_149811754,chr10_149835306,chr10_149921669,chr10_149921672,chr10_149928095,chr10_149928138,chr10_149928160,chr10_150087943
0,1,2369 x 3IIH6,2369,3IIH6,-0.027311,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70,1,2369 x PHN82,2369,PHN82,-0.254821,0.5,0.5,0.5,0.5,0.5,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,1,B37 x H95,B37,H95,0.003577,0.0,0.5,0.5,0.5,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210,1,B73 x MO17,B73,MO17,0.005360,0.5,0.5,0.5,1.5,0.5,...,1.0,0.0,0.5,0.0,0.0,1.0,0.0,1.0,0.0,0.0
280,1,B73 x PHM49,B73,PHM49,-0.196338,0.0,1.0,0.5,1.5,0.5,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [8]:
day=1
print(day)
df1 = pd.DataFrame(df[filter_days[day-1]])
X=df1.iloc[:,5:168391]
y=df1.iloc[:,4:5]
to_features = 1000
cor_support, cor_features, feature_index, corr_list = cor_selector(X,y['GrowthRate-Diff'], to_features)
feature_index.sort()
df_fs = pd.DataFrame({'Features':[X.columns[i]  for i in feature_index],'Pearson_cor':[corr_list[i] for i in feature_index]}, columns=['Features','Pearson_cor'])
des=df_fs.describe()
d=pd.DataFrame(des.T)
d=d.rename(index={'Pearson_cor': 'Day-'+str(day)})
print("Done")

for day in [i for i in range(2,70)]:
    print(day)
    df1 = pd.DataFrame(df[filter_days[day-1]])
    X=df1.iloc[:,5:168391]
    y=df1.iloc[:,4:5]
    cor_support, cor_features, feature_index, corr_list = cor_selector(X,y['GrowthRate-Diff'], to_features)
    feature_index.sort()
    df_fs = pd.DataFrame({'Features':[X.columns[i]  for i in feature_index],'Pearson_cor':[corr_list[i] for i in feature_index]}, columns=['Features','Pearson_cor'])
    des=df_fs.describe()
    td=pd.DataFrame(des.T)
    td=td.rename(index={'Pearson_cor': 'Day-'+str(day)})
    d=pd.DataFrame(d.append(td))
    print("Done")
    
d

1


//anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
//anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Done
2
Done
3
Done
4
Done
5
Done
6
Done
7
Done
8
Done
9
Done
10
Done
11
Done
12
Done
13
Done
14
Done
15
Done
16
Done
17
Done
18
Done
19
Done
20
Done
21
Done
22
Done
23
Done
24
Done
25
Done
26
Done
27
Done
28
Done
29
Done
30
Done
31
Done
32
Done
33
Done
34
Done
35
Done
36
Done
37
Done
38
Done
39
Done
40
Done
41
Done
42
Done
43
Done
44
Done
45
Done
46
Done
47
Done
48
Done
49
Done
50
Done
51
Done
52
Done
53
Done
54
Done
55
Done
56
Done
57
Done
58
Done
59
Done
60
Done
61
Done
62
Done
63
Done
64
Done
65
Done
66
Done
67
Done
68
Done
69
Done


,count,mean,std,min,25%,50%,75%,max
Day-1,1000.0,0.202446,0.185201,-0.315328,0.260781,0.262666,0.276516,0.387531
Day-2,1000.0,0.225662,0.164516,-0.305299,0.262736,0.270263,0.286474,0.409160
Day-3,1000.0,0.247074,0.131266,-0.286839,0.259806,0.273168,0.291528,0.418232
Day-4,1000.0,0.262155,0.091623,-0.286675,0.256899,0.272095,0.290649,0.415213
Day-5,1000.0,0.237846,0.133957,-0.296042,0.253533,0.265951,0.282091,0.400568
Day-6,1000.0,0.202490,0.175308,-0.294524,0.247966,0.258919,0.275280,0.382270
Day-7,1000.0,0.186262,0.191880,-0.299027,0.245893,0.255642,0.273546,0.371157
Day-8,1000.0,0.230931,0.150425,-0.299069,0.257709,0.266814,0.281258,0.369672
Day-9,1000.0,0.278552,0.077483,-0.293196,0.280916,0.280916,0.293000,0.375082
Day-10,1000.0,0.296186,0.043628,-0.284842,0.289905,0.296889,0.300589,0.387095


In [9]:
d.describe()

,count,mean,std,min,25%,50%,75%,max
count,69.0,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000
mean,1000.0,0.057319,0.182475,-0.323456,-0.032507,0.065702,0.155045,0.373264
std,0.0,0.219185,0.084228,0.137026,0.296184,0.288016,0.257535,0.050298
min,1000.0,-0.336227,0.011878,-0.427281,-0.370890,-0.364876,-0.347292,0.295504
25%,1000.0,-0.127870,0.123888,-0.382982,-0.294350,-0.274004,0.249839,0.322206
50%,1000.0,0.107562,0.189473,-0.355741,-0.255362,0.258919,0.281258,0.373784
75%,1000.0,0.266619,0.250545,-0.314409,0.280916,0.304485,0.312024,0.409160
max,1000.0,0.330646,0.322227,0.288454,0.344781,0.350616,0.371965,0.475296


In [13]:
for col in X.columns:
    X[col] = X[col].astype('category',copy=False)

In [14]:
X.dtypes

chr1_2111          category
chr1_6984          category
chr1_7035          category
chr1_7080          category
chr1_9405          category
chr1_10097         category
chr1_109083        category
chr1_109116        category
chr1_109133        category
chr1_109136        category
chr1_130363        category
chr1_222462        category
chr1_222473        category
chr1_222541        category
chr1_222588        category
chr1_224584        category
chr1_264849        category
chr1_267837        category
chr1_267845        category
chr1_267848        category
chr1_313280        category
chr1_319316        category
chr1_392144        category
chr1_458588        category
chr1_623826        category
chr1_623835        category
chr1_682678        category
chr1_709353        category
chr1_724669        category
chr1_766204        category
                     ...   
chr10_149596850    category
chr10_149596938    category
chr10_149597169    category
chr10_149597194    category
chr10_149597214    c

In [15]:
ux=np.unique(X.values)

In [16]:
ux

array([0.0, 0.5, 1.0, 1.5, 2.0], dtype=object)

In [18]:
day=1
print(day)
df1 = pd.DataFrame(df[filter_days[day-1]])
X=df1.iloc[:,5:168391]
y=df1.iloc[:,4:5]
to_features = 1000

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=to_features)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(to_features,'Score'))  #print 10 best features

1


ValueError: Unknown label type: (array([[-0.02731082],
       [-0.25482115],
       [ 0.00357746],
       [ 0.00536049],
       [-0.19633797],
       [ 0.0164739 ],
       [ 0.26812408],
       [-0.02272005],
       [-0.02479345],
       [ 0.06605535],
       [ 0.40194293],
       [-0.0532373 ],
       [ 0.23006603],
       [ 0.13829602],
       [ 0.01395197],
       [-0.26465634],
       [-0.01382511],
       [ 0.0175668 ],
       [ 0.0400091 ],
       [-0.00803926],
       [ 0.43631174],
       [-0.12076277],
       [ 0.02528901],
       [ 0.0385112 ],
       [ 0.15440966],
       [-0.18843935],
       [-0.03852356],
       [-0.00930471],
       [ 0.00085897],
       [-0.10455614],
       [ 0.00779406],
       [ 0.01786002],
       [ 0.18385686],
       [ 0.24310348],
       [ 0.21044581],
       [-0.28461217],
       [ 0.05066917],
       [ 0.09113805],
       [-0.02097402],
       [ 0.17687947],
       [ 0.04398068],
       [-0.13602681],
       [-0.42358075],
       [ 0.31122415],

In [70]:
df1 = pd.DataFrame(df[filter_days[0]])
X=df1.iloc[:,5:168391]
y=df1.iloc[:,4:5]
scaled_X = StandardScaler().fit_transform(X)
X_sdt = pd.DataFrame(scaled_X, columns=X.columns)
scaled_X = MinMaxScaler().fit_transform(X)
X_norm = pd.DataFrame(scaled_X, columns=X.columns)

In [34]:
## Pearson correlation based feature selection
to_features = 1000
cor_support, cor_features, feature_index, corr_list = cor_selector(X,y['GrowthRate-Diff'], to_features)
cor_support_sdt, cor_features_sdt, feature_index_sdt, corr_list_sdt = cor_selector(X_sdt,y['GrowthRate-Diff'], to_features)
cor_support_norm, cor_features_norm, feature_index_norm, corr_list_norm = cor_selector(X_norm,y['GrowthRate-Diff'], to_features)


//anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
//anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [49]:
feature_index.sort()
#df_fs = pd.DataFrame({'Features':[X.columns[i]  for i in feature_index],'Pearson_cor':[corr_list[i] for i in feature_index], 'Pearson_cor_sqr':[np.abs(corr_list[i]) for i in feature_index]}, columns=['Features','Pearson_cor','Pearson_cor_sqr'])
df_fs = pd.DataFrame({'Features':[X.columns[i]  for i in feature_index],'Pearson_cor':[corr_list[i] for i in feature_index]}, columns=['Features','Pearson_cor'])
#df_fs.sort_values(by='Features', ascending=False, inplace=True)
des=df_fs.describe()

In [52]:
d=pd.DataFrame(des.T)
d=d.rename(index={'Pearson_cor': 'Day-'+str(1)})
d

,count,mean,std,min,25%,50%,75%,max
Day-1,1000.0,0.202446,0.185201,-0.315328,0.260781,0.262666,0.276516,0.387531


In [36]:
feature_index.sort()
df_fs_std = pd.DataFrame({'Features':[X_sdt.columns[i]  for i in feature_index_sdt],'Pearson_cor':[corr_list_sdt[i] for i in feature_index_sdt], 'Pearson_cor_sqr':[np.abs(corr_list_sdt[i]) for i in feature_index]}, columns=['Features','Pearson_cor','Pearson_cor_sqr'])
#df_fs_std.sort_values(by='Pearson_cor_sqr', ascending=False, inplace=True)
df_fs_std.describe()



,Pearson_cor,Pearson_cor_sqr
count,1000.000000,1000.000000
mean,0.202446,0.273830
std,0.185201,0.016333
min,-0.315328,0.260781
25%,0.260781,0.261637
50%,0.262666,0.267240
75%,0.276516,0.280911
max,0.387531,0.387531


In [37]:
feature_index.sort()
df_fs_norm = pd.DataFrame({'Features':[X_norm.columns[i]  for i in feature_index_norm],'Pearson_cor':[corr_list_norm[i] for i in feature_index_norm], 'Pearson_cor_sqr':[np.abs(corr_list_norm[i]) for i in feature_index]}, columns=['Features','Pearson_cor','Pearson_cor_sqr'])
#df_fs_norm.sort_values(by='Pearson_cor_sqr', ascending=False, inplace=True)
df_fs_norm.describe()


,Pearson_cor,Pearson_cor_sqr
count,1000.000000,1000.000000
mean,0.202446,0.273830
std,0.185201,0.016333
min,-0.315328,0.260781
25%,0.260781,0.261637
50%,0.262666,0.267240
75%,0.276516,0.280911
max,0.387531,0.387531


In [60]:
model = SelectKBest(score_func=f_regression, k=100)
results = model.fit(X, df['qsec'])

print results.scores_
print results.pvalues_

TypeError: __init__() got an unexpected keyword argument 'penalty'